In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
%cd "/gdrive/My Drive"

/gdrive/My Drive


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.base import clone
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import math

from tqdm import tqdm
import joblib
import gc

## config

In [0]:
INPUT = './analysis/mole/data/raw/'
TRAIN_PATH = INPUT + 'train.csv'
TEST_PATH = INPUT + 'test.csv'
PREPROCESS = './analysis/mole/data/preprocess/'

MID_MODEL_PATH = PREPROCESS + 'middle_model.pkl'
MODEL_PATH = PREPROCESS + 'model.pkl'
ENCODER_PATH = PREPROCESS + 'le.pkl'

RUN_PLOT = True
TARGET = 'scalar_coupling_constant'
N_FOLDS = 3

atom_weight = {'H': 1.008, 'C': 12.01, 'N': 14.01, 'O':16.00}

## logging

In [0]:
import logging
import logging.handlers


def create_logger(log_file_name):
    logger_ = logging.getLogger('main')
    logger_.setLevel(logging.DEBUG)
    fh = logging.handlers.RotatingFileHandler(
        log_file_name, maxBytes=100000, backupCount=8)
    fh.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        '[%(levelname)s]%(asctime)s:%(name)s:%(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    # add the handlers to the logger
    logger_.addHandler(fh)
    logger_.addHandler(ch)


def get_logger():
    return logging.getLogger('main')

In [0]:
create_logger('mole.log')

## util

In [0]:
def onehot(_df):
    cat_names = [name for name, col in _df.iteritems() if col.dtype == 'O']
    df_cat = pd.get_dummies(_df[cat_names])
    _df = pd.concat([_df, df_cat], axis=1).drop(cat_names, axis=1)
    return _df

def label_encode(df):
    cat_names = [name for name, col in df.iteritems() if col.dtype == 'O']    
    for cat_name in cat_names:
        print(cat_name)
        le = LabelEncoder()
        le.fit(df[cat_name].values)
        df[cat_name] = le.transform(df[cat_name].values)
    return df

class Encoder:
    def __init__(self):        
        self.encoders = {}
    
    def fit(self, df, cat_names):
        for cat_name in cat_names:
            le = LabelEncoder()
            le.fit(df[cat_name].values)
            self.encoders[cat_name] = le        
    
    def transform(self, df):
        for cat_name in self.encoders.keys():            
            df[cat_name] = self.encoders[cat_name].transform(df[cat_name].values)
            
        return df


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    
    return df

## Preprocess

In [0]:
def map_atom_info(df, strct, atom_idx):
    df = pd.merge(df, strct, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

def calc_dist(df):
    p_0 = df[['x_0', 'y_0', 'z_0']].values
    p_1 = df[['x_1', 'y_1', 'z_1']].values

    df['dist'] = np.linalg.norm(p_0 - p_1, axis=1)
    df['dist_x'] = (df['x_0'] - df['x_1']) ** 2
    df['dist_y'] = (df['y_0'] - df['y_1']) ** 2
    df['dist_z'] = (df['z_0'] - df['z_1']) ** 2

    return df

def divide_type(df):    
    df['type_0'] = df['type'].apply(lambda x: x[0])
    df['type_1'] = df['type'].apply(lambda x: x[1:])
    return df

In [0]:
def feature_engineering(df):
    print("Starting Feature Engineering...")
    g = df.groupby('molecule_name')
    g1 = df.groupby(['molecule_name', 'atom_index_0'])
    g2 = df.groupby(['molecule_name', 'atom_index_1'])
    g3 = df.groupby(['molecule_name', 'atom_1'])
    g4 = df.groupby(['molecule_name', 'type_0'])
    g5 = df.groupby(['molecule_name', 'type'])
    
    df['type_0'] = df['type'].apply(lambda x: x[0])
    df['molecule_couples'] = g['id'].transform('count')
    df['molecule_dist_mean'] = g['dist'].transform('mean')
    df['molecule_dist_min'] = g['dist'].transform('min')
    df['molecule_dist_max'] = g['dist'].transform('max')
    df['atom_0_couples_count'] = g1['id'].transform('count')
    df['atom_1_couples_count'] = g2['id'].transform('count')
    df[f'molecule_atom_index_0_x_1_std'] = g1['x_1'].transform('std')
    df[f'molecule_atom_index_0_y_1_mean'] = g1['y_1'].transform('mean')
    df[f'molecule_atom_index_0_y_1_mean_diff'] = df[f'molecule_atom_index_0_y_1_mean'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_mean_div'] = df[f'molecule_atom_index_0_y_1_mean'] / df['y_1']
    df[f'molecule_atom_index_0_y_1_max'] = g1['y_1'].transform('max')
    df[f'molecule_atom_index_0_y_1_max_diff'] = df[f'molecule_atom_index_0_y_1_max'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_std'] = g1['y_1'].transform('std')
    df[f'molecule_atom_index_0_z_1_std'] = g1['z_1'].transform('std')
    df[f'molecule_atom_index_0_dist_mean'] = g1['dist'].transform('mean')
    df[f'molecule_atom_index_0_dist_mean_diff'] = df[f'molecule_atom_index_0_dist_mean'] - df['dist']
    df[f'molecule_atom_index_0_dist_mean_div'] = df[f'molecule_atom_index_0_dist_mean'] / df['dist']
    df[f'molecule_atom_index_0_dist_max'] = g1['dist'].transform('max')
    df[f'molecule_atom_index_0_dist_max_diff'] = df[f'molecule_atom_index_0_dist_max'] - df['dist']
    df[f'molecule_atom_index_0_dist_max_div'] = df[f'molecule_atom_index_0_dist_max'] / df['dist']
    df[f'molecule_atom_index_0_dist_min'] = g1['dist'].transform('min')
    df[f'molecule_atom_index_0_dist_min_diff'] = df[f'molecule_atom_index_0_dist_min'] - df['dist']
    df[f'molecule_atom_index_0_dist_min_div'] = df[f'molecule_atom_index_0_dist_min'] / df['dist']
    df[f'molecule_atom_index_0_dist_std'] = g1['dist'].transform('std')
    df[f'molecule_atom_index_0_dist_std_diff'] = df[f'molecule_atom_index_0_dist_std'] - df['dist']
    df[f'molecule_atom_index_0_dist_std_div'] = df[f'molecule_atom_index_0_dist_std'] / df['dist']
    df[f'molecule_atom_index_1_dist_mean'] = g2['dist'].transform('mean')
    df[f'molecule_atom_index_1_dist_mean_diff'] = df[f'molecule_atom_index_1_dist_mean'] - df['dist']
    df[f'molecule_atom_index_1_dist_mean_div'] = df[f'molecule_atom_index_1_dist_mean'] / df['dist']
    df[f'molecule_atom_index_1_dist_max'] = g2['dist'].transform('max')
    df[f'molecule_atom_index_1_dist_max_diff'] = df[f'molecule_atom_index_1_dist_max'] - df['dist']
    df[f'molecule_atom_index_1_dist_max_div'] = df[f'molecule_atom_index_1_dist_max'] / df['dist']
    df[f'molecule_atom_index_1_dist_min'] = g2['dist'].transform('min')
    df[f'molecule_atom_index_1_dist_min_diff'] = df[f'molecule_atom_index_1_dist_min'] - df['dist']
    df[f'molecule_atom_index_1_dist_min_div'] = df[f'molecule_atom_index_1_dist_min'] / df['dist']
    df[f'molecule_atom_index_1_dist_std'] = g2['dist'].transform('std')
    df[f'molecule_atom_index_1_dist_std_diff'] = df[f'molecule_atom_index_1_dist_std'] - df['dist']
    df[f'molecule_atom_index_1_dist_std_div'] = df[f'molecule_atom_index_1_dist_std'] / df['dist']
    df[f'molecule_atom_1_dist_mean'] = g3['dist'].transform('mean')
    df[f'molecule_atom_1_dist_min'] = g3['dist'].transform('min')
    df[f'molecule_atom_1_dist_min_diff'] = df[f'molecule_atom_1_dist_min'] - df['dist']
    df[f'molecule_atom_1_dist_min_div'] = df[f'molecule_atom_1_dist_min'] / df['dist']
    df[f'molecule_atom_1_dist_std'] = g3['dist'].transform('std')
    df[f'molecule_atom_1_dist_std_diff'] = df[f'molecule_atom_1_dist_std'] - df['dist']
    df[f'molecule_type_0_dist_std'] = g4['dist'].transform('std')
    df[f'molecule_type_0_dist_std_diff'] = df[f'molecule_type_0_dist_std'] - df['dist']
    df[f'molecule_type_dist_mean'] = g5['dist'].transform('mean')
    df[f'molecule_type_dist_mean_diff'] = df[f'molecule_type_dist_mean'] - df['dist']
    df[f'molecule_type_dist_mean_div'] = df[f'molecule_type_dist_mean'] / df['dist']
    df[f'molecule_type_dist_max'] = g5['dist'].transform('max')
    df[f'molecule_type_dist_min'] = g5['dist'].transform('min')
    df[f'molecule_type_dist_std'] = g5['dist'].transform('std')
    df[f'molecule_type_dist_std_diff'] = df[f'molecule_type_dist_std'] - df['dist']

    # TODO: back
    # df = reduce_mem_usage(df)
    
    return df

In [0]:
def add_1j(df):
    get_logger().info('load df_1j')
    
    df_1j = joblib.load(PREPROCESS + 'df_1j.pkl')
    
    df = df.merge(df_1j, on=['molecule_name', 'atom_index_0', 'atom_index_1'], how='left') 
    
    return df


def add_2j_center_atom(df):    
    get_logger().info('load df_2jsim')
    
    df_2j = joblib.load(PREPROCESS + 'df_2jsim.pkl')  
    
    # atom weight
    df_2j['2j_atom_center_weight'] = df_2j['2j_atom_center'].replace(atom_weight)
    
    # sum of norm
    df_2j['2j_sum_norm_vec'] = df_2j['2j_norm_vec_02'] + df_2j['2j_norm_vec_12']
    
    df = df.merge(df_2j, on=['molecule_name', 'atom_index_0', 'atom_index_1'], how='left')    
    
    # replace missing vlaue to 'nan' for LabelEncoder
    df.loc[df['2j_atom_center'].isnull(), '2j_atom_center'] = 'nan'
    
    return df

def str_sort(s):
    """
    Parameters
    ----------
    x: str   
    """
    # print(s)
    if not isinstance(s, str):
        return s
    elif s[0] > s[1]:
        return s[1] + s[0]
    else:
        return s

def add_3j_center_atom(df):    
    get_logger().info('load df_3jsim')
    
    df_3j = joblib.load(PREPROCESS + 'df_3jsim.pkl')
    
    # atom weight
    s_atom_w0 = df_3j['3j_atom_center_0'].replace(atom_weight)
    s_atom_w1 = df_3j['3j_atom_center_1'].replace(atom_weight)
    df_3j['3j_atom_center_weight'] = s_atom_w0 + s_atom_w1

    # concatenate atom string 'C' + 'C' - > 'CC'
    tmp = df_3j['3j_atom_center_0'] + df_3j['3j_atom_center_1']
    df_3j['3j_atom_center'] = tmp.transform(str_sort)    
    df_3j.drop(['3j_atom_center_0', '3j_atom_center_1'], axis=1, inplace=True)
    
    # sum norm_vec
    df_3j['3j_sum_norm_vec'] = df_3j['3j_norm_vec_02'] + df_3j['3j_norm_vec_13'] + df_3j['3j_norm_vec_23']
    
    df = df.merge(df_3j, on=['molecule_name', 'atom_index_0', 'atom_index_1'], how='left')    
    
    # replace missing vlaue to 'nan' for LabelEncoder
    df.loc[df['3j_atom_center'].isnull(), '3j_atom_center'] = 'nan'    
    
    return df

In [0]:
def drop_col(df_org):
    df = df_org.copy()
    to_drop = ['id', 'molecule_name', 'atom_index_0', 'atom_index_1',
               'x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', #'dist_x', 'dist_y', 'dist_z',
               'atom_0', 'atom_1'
              ]
    df = df.drop(to_drop, axis=1)
    
    return df

In [0]:
def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    """
    Fast metric computation for this competition: https://www.kaggle.com/c/champs-scalar-coupling
    Code is from this kernel: https://www.kaggle.com/uberkinder/efficient-metric
    """
    maes = (y_true-y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

def oof_train(_X, _y, _types):
    """
    Parameters
    ----------
    _X: pd.DataFrame, shape [n_samples, n_features]
    _y: array-like object, shape [n_samples]
    _types: array-like object, shsape [n_samples]
        array of `type` (e.g. 2JHC, 1JHC, 3JHH, etc.)
    """
    # TODO: divide data to training and validation about molecular
    
    models = []
    df_scores = pd.DataFrame(columns=['valid_score'])
    df_pred = pd.DataFrame(index=_X.index).reset_index(drop=True)

    fold = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1)
    for n_fold, (train_idx, valid_idx) in enumerate(fold.split(_X, _types)):
        # prepare data
        X_train, y_train = _X.iloc[train_idx], _y.iloc[train_idx]
        X_valid, y_valid = _X.iloc[valid_idx], _y.iloc[valid_idx]
        print('mean of target. train:{}, valid:{}'.format(y_train.mean(), y_valid.mean()))

        # generate model
        model = gen_model(_X)
        
        # train
        model.fit(X_train, y_train, eval_metric='mae',
                  eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=100,
                  early_stopping_rounds=100
                  )
        
        # validate
        y_pred = model.predict(X_valid, num_iteration=model.best_iteration_)
        
        types_valid = _types.iloc[valid_idx]
        valid_score = group_mean_log_mae(y_valid, y_pred, types_valid)
        get_logger().info('fold %d valid %f' % (n_fold+1, valid_score))
        
        df_scores = df_scores.append(pd.Series([valid_score], index=['valid_score']), ignore_index=True)
        df_pred.loc[valid_idx, 'proba'] = y_pred
        df_pred.loc[valid_idx, 'y_true'] = y_valid
        models.append(model)
        
        # TODO: back
        # break
    get_logger().info('CV score: %f' % df_scores.mean()[0])
    
    return models, df_scores, df_pred

def oof_predict(_models, _X):
    get_logger().info('Start oof_predict')
    y_pred = np.zeros(_X.shape[0])
        
    for i, model in enumerate(_models):
        get_logger().info('prediction: %d' % i)
        y_pred += model.predict(_X) / len(_models)
    
    get_logger().info('Finish oof_predict')
    return y_pred


def gen_model(_X):
    n_features = _X.shape[1]
    colsample_rate = max(0.7, math.sqrt(n_features)/n_features)
    
    _model = lgb.LGBMRegressor(
        learning_rate=0.2,
        n_estimators=1500,
        num_leaves=128,
        # min_child_weight=15, # good value: 0, 5, 15, 300
        min_child_samples=80,
        subsample=0.7,
        colsample_bytree=1, #colsample_rate,
        objective='regression',
        reg_lambda=0.1,
        reg_alpha=0.1,
        seed=2019
        )
    return _model


In [0]:
def preprocess(df, strct, mode, s_type=None):
    """
    Parameters
    ----------
    df: pd.DataFrame
        dataframe of train.csv or test.csv
    strct: pd.DataFrame
        dataframe of structures.csv
    mode: str
        'train' or 'predict'
    s_type: None or pd.Series
        'type' column (e.g. 1JHC, 2JHH).
        If mode is 'train', the s_type must be specified.
    """
    get_logger().info('Start preprocess()')
    df = add_1j(df)
    df = add_2j_center_atom(df)
    df = add_3j_center_atom(df)
    df = map_atom_info(df, strct, 0)
    df = map_atom_info(df, strct, 1)
    df = calc_dist(df)
    df = divide_type(df)
    df = feature_engineering(df)
    
    display(df.head(10))
    display(df.tail(10))
    
    # encode
    if mode == 'train':
        enc = Encoder()
        enc.fit(df, ['type', 'type_0', 'type_1', 
                     '2j_atom_center', '3j_atom_center'])
        joblib.dump(enc, ENCODER_PATH)
    elif mode == 'predict':
        get_logger().info('loading encoder from %s' % ENCODER_PATH)
        enc = joblib.load(ENCODER_PATH)
    df = enc.transform(df)
        
    use_features = [col for col in df.columns if col not in [TARGET]]
    get_logger().info(use_features)
    df[use_features] = reduce_mem_usage(df[use_features])
    # TODO: back
    # df = add_scc_feature(df, 'fc', mode=mode, s_type=s_type)
    
    get_logger().info('Finish preprocess()')
    return df

In [0]:
def drop_uneffect_feature(df):
    """
    Drop uneffective features from dataframe
    """
    for col in df.columns:
        if len(df[col].unique()) == 1:
            df.drop(col, axis=1, inplace=True)
    return df

### fermi constant

In [0]:
class CNTR:
    """Model to predict fc/sd/pso/dso columns"""
    
    def __init__(self, y_col):
        self.y_col = y_col
        
    def train(self, df_org, scc, s_type):
        """
        Parameters
        ----------
        s_type: pd.Series
            'type' column (e.g. 1JHC, 2JHH)
        """
        df = df_org.copy()
        # Merge
        key_cols = ['molecule_name', 'atom_index_0', 'atom_index_1']
        df = df.merge(scc[key_cols + [self.y_col]], how='left', on=key_cols)
        
        # drop unnecessary cols        
        df = drop_col(df)        
        
        y = df[self.y_col].copy()        
        df.drop([TARGET, self.y_col], axis=1, inplace=True)
        X = df
        
        display(X.head())
        display(y.head())
        models, scores, y_pred = oof_train(X, y, s_type)
        
        # save model
        joblib.dump(models, MID_MODEL_PATH)
        
        self.models_ = models
        self.scores_ = scores
        self.y_pred_ = y_pred
        
    def predict(self, df_org):    
        y_pred = np.zeros(df_org.shape[0])
        
        X = df_org.copy()
        X = drop_col(X)
        
        display(X.head())
        # X = self.preprocess(df_org)
        for model in self.models_:            
            y_pred += model.predict(X) / len(models)
        
        return y_pred
    
    def load_model(self):
        # load pkl by joblib
        self.models_ = joblib.load(MID_MODEL_PATH)

In [0]:
def add_scc_feature(df, cntr_name, mode, s_type=None):
    """
    Parameters
    ----------
    cntr_name: str
        'fc', 'sd', 'pso' or 'dso'
    mode: str
        'train' or 'predict'
    s_type: None or pd.Series
        'type' column (e.g. 1JHC, 2JHH).
        If mode is 'train', the s_type must be specified.
    """
    add_feature = '%s_pred' % cntr_name
    cntr = CNTR(cntr_name)
    if mode == 'train': 
        assert s_type is not None, 's_type must be specified.'
        
        get_logger().info('start loading scalar_coupling_contributions')
        scc = pd.read_csv(INPUT + 'scalar_coupling_contributions.csv')
        get_logger().info('finished loading scalar_coupling_contributions')
        
        # train contribution(fc/sd/pso/dso)
        cntr.train(df, scc, s_type)
    
        display(cntr.y_pred_.head())
        df[add_feature] = cntr.y_pred_
    elif mode == 'predict':
        cntr.load_model()
        y_pred = cntr.predict(df)
        df[add_feature] = y_pred
    
    return df

## Train

In [0]:
df_train = pd.read_csv(TRAIN_PATH)
df_strct = pd.read_csv(INPUT + 'structures.csv')

In [0]:
# TODO: remove
# df_train = df_train[(df_train['type']=='1JHC') | (df_train['type']=='1JHN')]

In [0]:
def train_single_model(df, strct):
    # TODO: back
    df = df.head(10000)

    s_type = df['type'].copy()

    df = preprocess(df, strct, mode='train', s_type=s_type)
    df = drop_col(df)

    y = df[TARGET].copy()
    df.drop([TARGET], axis=1, inplace=True)
    X = df
    
    display(X.head())
    display(y.head())
    models, df_scores, df_pred = oof_train(X, y, s_type)

    joblib.dump(models, MODEL_PATH)
    
    return models, df_scores, df_pred

In [0]:
def train_models_each_type(df, strct):
    # TODO:back
    # df = df.head(10000)
    
    s_type = df['type'].copy()
    
    df = preprocess(df, strct, mode='train', s_type=s_type)
    df = drop_col(df)
    
    model_dict = {}
    score_dict = {}
    pred_dict = {}
    coupling_types = s_type.unique()
    for coup_type in coupling_types:
        get_logger().info('Starting train model(%s)' % coup_type)
        is_the_type = (s_type == coup_type)        
        df_type = df[is_the_type.values]
                
        y = df_type[TARGET]
        df_type.drop([TARGET], axis=1, inplace=True)
        X = df_type
        X = drop_uneffect_feature(X)
        
        get_logger().info('features(%s): %s' % (coup_type, str(X.columns.tolist())))
        display(X.head())
        display(y.head())
        models, df_scores, df_pred = oof_train(X, y, _types=s_type[is_the_type].reset_index(drop=True))
        
        model_dict[coup_type] = models
        score_dict[coup_type] = df_scores
        pred_dict[coup_type] = df_pred
        
    joblib.dump(model_dict, MODEL_PATH)
    
    return model_dict, score_dict, pred_dict

In [0]:
# models, df_scores, df_pred = train_single_model(df_train, df_strct)
model_dict, score_dict, pred_dict = train_models_each_type(df_train, df_strct)

[INFO]2019-06-25 15:07:25,997:main:Start preprocess()
[INFO]2019-06-25 15:07:26,249:main:load df_1j
[INFO]2019-06-25 15:07:30,410:main:load df_2jsim
[INFO]2019-06-25 15:07:40,693:main:load df_3jsim


Starting Feature Engineering...


,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,1j_nbonds,2j_atom_center,2j_area_021,2j_norm_vec_02,2j_norm_vec_12,2j_cos,2j_atom_center_weight,2j_sum_norm_vec,3j_norm_vec_02,3j_norm_vec_13,3j_norm_vec_23,3j_cos_023,3j_cos_231,3j_area_023,3j_area_231,3j_dihedral,3j_atom_center_weight,3j_atom_center,3j_sum_norm_vec,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,dist_x,dist_y,dist_z,type_0,type_1,molecule_couples,...,molecule_atom_index_0_z_1_std,molecule_atom_index_0_dist_mean,molecule_atom_index_0_dist_mean_diff,molecule_atom_index_0_dist_mean_div,molecule_atom_index_0_dist_max,molecule_atom_index_0_dist_max_diff,molecule_atom_index_0_dist_max_div,molecule_atom_index_0_dist_min,molecule_atom_index_0_dist_min_diff,molecule_atom_index_0_dist_min_div,molecule_atom_index_0_dist_std,molecule_atom_index_0_dist_std_diff,molecule_atom_index_0_dist_std_div,molecule_atom_index_1_dist_mean,molecule_atom_index_1_dist_mean_diff,molecule_atom_index_1_dist_mean_div,molecule_atom_index_1_dist_max,molecule_atom_index_1_dist_max_diff,molecule_atom_index_1_dist_max_div,molecule_atom_index_1_dist_min,molecule_atom_index_1_dist_min_diff,molecule_atom_index_1_dist_min_div,molecule_atom_index_1_dist_std,molecule_atom_index_1_dist_std_diff,molecule_atom_index_1_dist_std_div,molecule_atom_1_dist_mean,molecule_atom_1_dist_min,molecule_atom_1_dist_min_diff,molecule_atom_1_dist_min_div,molecule_atom_1_dist_std,molecule_atom_1_dist_std_diff,molecule_type_0_dist_std,molecule_type_0_dist_std_diff,molecule_type_dist_mean,molecule_type_dist_mean_diff,molecule_type_dist_mean_div,molecule_type_dist_max,molecule_type_dist_min,molecule_type_dist_std,molecule_type_dist_std_diff
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,4.0,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953,0.000220,1.192105,0.000036,1,JHC,10,...,0.727957,1.610344,0.518391,1.474738,1.783157,0.691204,1.632998,1.091953,0.000000,1.000000,0.345594,-0.746359,0.316492,1.091950,-0.000003,0.999997,1.091953,0.000000,1.000000,1.091946,-6.680479e-06,0.999994,0.000003,-1.091950,0.000003,1.091950,1.091946,-0.000007,0.999994,0.000003,-1.091950,0.000003,-1.091950,1.091950,-0.000003,0.999997,1.091953,1.091946,0.000003,-1.091950
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,NaN,C,1.124187,1.091953,1.091952,-0.333287,12.01,2.183905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120,1.019253,2.160261,0.000003,2,JHH,10,...,0.727957,1.610344,-0.172776,0.903105,1.783157,0.000037,1.000021,1.091953,-0.691167,0.612383,0.345594,-1.437526,0.193814,1.783120,0.000000,1.000000,1.783120,0.000000,1.000000,1.783120,0.000000e+00,1.000000,NaN,NaN,NaN,1.783146,1.783120,0.000000,1.000000,0.000014,-1.783106,0.000014,-1.783106,1.783146,0.000027,1.000015,1.783158,1.783120,0.000014,-1.783106
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,NaN,C,1.124162,1.091953,1.091946,-0.333335,12.01,2.183899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147,0.294812,2.112831,0.771973,2,JHH,10,...,0.727957,1.610344,-0.172803,0.903091,1.783157,0.000009,1.000005,1.091953,-0.691194,0.612374,0.345594,-1.437553,0.193811,1.783153,0.000005,1.000003,1.783158,0.000010,1.000006,1.783147,0.000000e+00,1.000000,0.000007,-1.783140,0.000004,1.783146,1.783120,-0.000028,0.999984,0.000014,-1.783134,0.000014,-1.783134,1.783146,-0.000001,0.999999,1.783158,1.783120,0.000014,-1.783134
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,NaN,C,1.124158,1.091953,1.091948,-0.333347,12.01,2.183901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,1.783157,0.276638,2.085032,0.817978,2,JHH,10,...,0.727957,1.610344,-0.172812,0.903086,1.783157,0.000000,1.000000,1.091953,-0.691204,0.612371,0.345594,-1.437563,0.193810,1.783151,-0.000006,0.999997,1.783157,0.000000,1.000000,1.783148,-8.813100e-06,0.999995,0.000005,-1.783152,0.000003,

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,1j_nbonds,2j_atom_center,2j_area_021,2j_norm_vec_02,2j_norm_vec_12,2j_cos,2j_atom_center_weight,2j_sum_norm_vec,3j_norm_vec_02,3j_norm_vec_13,3j_norm_vec_23,3j_cos_023,3j_cos_231,3j_area_023,3j_area_231,3j_dihedral,3j_atom_center_weight,3j_atom_center,3j_sum_norm_vec,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,dist_x,dist_y,dist_z,type_0,type_1,molecule_couples,...,molecule_atom_index_0_z_1_std,molecule_atom_index_0_dist_mean,molecule_atom_index_0_dist_mean_diff,molecule_atom_index_0_dist_mean_div,molecule_atom_index_0_dist_max,molecule_atom_index_0_dist_max_diff,molecule_atom_index_0_dist_max_div,molecule_atom_index_0_dist_min,molecule_atom_index_0_dist_min_diff,molecule_atom_index_0_dist_min_div,molecule_atom_index_0_dist_std,molecule_atom_index_0_dist_std_diff,molecule_atom_index_0_dist_std_div,molecule_atom_index_1_dist_mean,molecule_atom_index_1_dist_mean_diff,molecule_atom_index_1_dist_mean_div,molecule_atom_index_1_dist_max,molecule_atom_index_1_dist_max_diff,molecule_atom_index_1_dist_max_div,molecule_atom_index_1_dist_min,molecule_atom_index_1_dist_min_diff,molecule_atom_index_1_dist_min_div,molecule_atom_index_1_dist_std,molecule_atom_index_1_dist_std_diff,molecule_atom_index_1_dist_std_div,molecule_atom_1_dist_mean,molecule_atom_1_dist_min,molecule_atom_1_dist_min_diff,molecule_atom_1_dist_min_div,molecule_atom_1_dist_std,molecule_atom_1_dist_std_diff,molecule_type_0_dist_std,molecule_type_0_dist_std_diff,molecule_type_dist_mean,molecule_type_dist_mean_diff,molecule_type_dist_mean_div,molecule_type_dist_max,molecule_type_dist_min,molecule_type_dist_std,molecule_type_dist_std_diff
4658137,4658137,dsgdb9nsd_133884,16,8,2JHC,9.119730,NaN,C,1.419871,1.090480,1.509930,-0.506344,12.01,2.600410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,-0.084531,1.110807,-1.796741,C,0.787756,-0.840138,-1.042152,2.266379,0.760884,3.806185,0.569405,2,JHC,78,...,0.916944,2.600236,0.333857,1.147309,3.358747,1.092368,1.481988,1.090480,-1.175899,0.481155,0.679559,-1.586820,0.299844,2.536801,0.270422,1.119319,3.448811,1.182432,1.521728,1.080997,-1.185382,0.476971,0.823964,-1.442415,0.363560,2.625259,1.080997,-1.185382e+00,0.476971,0.794133,-1.472246,0.135944,-2.130435,2.274912,0.008533,1.003765,2.312206,2.209489,0.037188,-2.229191
4658138,4658138,dsgdb9nsd_133884,16,17,3JHH,0.789559,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1.090480,1.080997,1.509930,-0.506344,-0.581363,1.419871,1.328055,0.823388,24.02,CC,3.681407,H,-0.084531,1.110807,-1.796741,H,1.126550,-1.348733,-1.933838,2.744968,1.466716,6.049335,0.018796,3,JHH,78,...,0.916944,2.600236,-0.144732,0.947274,3.358747,0.613780,1.223602,1.090480,-1.654488,0.397265,0.679559,-2.065408,0.247565,2.705595,-0.039372,0.985657,2.744968,0.000000,1.000000,2.662406,-0.082561,0.969923,0.041413,-2.703555,0.015087,2.503545,1.784424,-9.605434e-01,0.650071,0.403132,-2.341835,0.344874,-2.400093,2.706152,-0.038816,0.985859,2.744969,2.662400,0.033847,-2.711121
4658139,4658139,dsgdb9nsd_133884,17,1,3JHN,-0.006537,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1.080997,1.560647,1.509930,-0.581363,-0.411704,1.328055,2.147490,-0.310911,24.02,CC,4.151574,H,1.126550,-1.348733,-1.933838,N,-1.571100,0.047932,-0.491726,3.362689,7.277316,1.950674,2.079688,3,JHN,78,...,0.755962,2.692607,-0.670082,0.800730,3.495226,0.132537,1.039414,1.080997,-2.281692,0.321468,0.845189,-2.517500,0.251343,2.609557,-0.753132,0.776033,3.362702,0.000013,1.000004,2.220388,-1.142301,0.660301,0.583487,-2.779202,0.173518,2.609557,2.220388,-1.142301e+00,0.660301,0.583487,-2.779202,0.344874,-3.017815,3.362695,0.000006,1.000002,3.362702,3.362689,0.000009,-3.362680
4658140,4658140,dsgdb9nsd_133884,17,2,3JHC,1.944380,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1.080997,1.509931,1.509711,-0.554904,-0.171603,1.357679,2.245745,-0.825532,24.02,CC,4.100640,H,1.126550,-1.348733,-1.933838,C,-0.750850,-0.602182,0.665932,3.292533,3.524629,0.557339,6.758808,3,JHC,78,...,0.755962,2.692607,-0.599926,0.817792,3.495226,0.202693,1.0

[INFO]2019-06-25 15:08:36,393:main:['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type', '1j_nbonds', '2j_atom_center', '2j_area_021', '2j_norm_vec_02', '2j_norm_vec_12', '2j_cos', '2j_atom_center_weight', '2j_sum_norm_vec', '3j_norm_vec_02', '3j_norm_vec_13', '3j_norm_vec_23', '3j_cos_023', '3j_cos_231', '3j_area_023', '3j_area_231', '3j_dihedral', '3j_atom_center_weight', '3j_atom_center', '3j_sum_norm_vec', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'dist', 'dist_x', 'dist_y', 'dist_z', 'type_0', 'type_1', 'molecule_couples', 'molecule_dist_mean', 'molecule_dist_min', 'molecule_dist_max', 'atom_0_couples_count', 'atom_1_couples_count', 'molecule_atom_index_0_x_1_std', 'molecule_atom_index_0_y_1_mean', 'molecule_atom_index_0_y_1_mean_diff', 'molecule_atom_index_0_y_1_mean_div', 'molecule_atom_index_0_y_1_max', 'molecule_atom_index_0_y_1_max_diff', 'molecule_atom_index_0_y_1_std', 'molecule_atom_index_0_z_1_std', 'molecule_atom_index_0_dist_mean', 'molecul

...

[INFO]2019-06-25 16:45:50,318:main:fold 1 valid -2.115853


mean of target. train:0.9900282797472645, valid:0.9921330153410689
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 0.0665362	training's l1: 0.17141	valid_1's l2: 0.123547	valid_1's l1: 0.219289
[200]	training's l2: 0.038311	training's l1: 0.131967	valid_1's l2: 0.109901	valid_1's l1: 0.203993
[300]	training's l2: 0.0248201	training's l1: 0.107769	valid_1's l2: 0.103346	valid_1's l1: 0.196504
[400]	training's l2: 0.0170714	training's l1: 0.0903531	valid_1's l2: 0.0999268	valid_1's l1: 0.192573
[500]	training's l2: 0.0122111	training's l1: 0.0770191	valid_1's l2: 0.0978756	valid_1's l1: 0.189842
[600]	training's l2: 0.00903547	training's l1: 0.0664369	valid_1's l2: 0.0965752	valid_1's l1: 0.188101
[700]	training's l2: 0.00663286	training's l1: 0.0574072	valid_1's l2: 0.0954158	valid_1's l1: 0.186584
[800]	training's l2: 0.00503176	training's l1: 0.0501613	valid_1's l2: 0.0946469	valid_1's l1: 0.185579
[900]	training's l2: 0.00383359	training's l1: 0.04

[INFO]2019-06-25 16:47:58,949:main:fold 2 valid -2.066710


mean of target. train:0.9908702216613917, valid:0.990449139099707
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 0.0696875	training's l1: 0.173641	valid_1's l2: 0.12797	valid_1's l1: 0.22157
[200]	training's l2: 0.0397558	training's l1: 0.132914	valid_1's l2: 0.112668	valid_1's l1: 0.205007
[300]	training's l2: 0.0253475	training's l1: 0.108076	valid_1's l2: 0.106301	valid_1's l1: 0.197904
[400]	training's l2: 0.0173393	training's l1: 0.0905601	valid_1's l2: 0.102934	valid_1's l1: 0.194004
[500]	training's l2: 0.0122701	training's l1: 0.0769351	valid_1's l2: 0.100554	valid_1's l1: 0.191067
[600]	training's l2: 0.00893245	training's l1: 0.066145	valid_1's l2: 0.0991038	valid_1's l1: 0.189118
[700]	training's l2: 0.00657579	training's l1: 0.05726	valid_1's l2: 0.0979407	valid_1's l1: 0.187826
[800]	training's l2: 0.00496125	training's l1: 0.049953	valid_1's l2: 0.0972266	valid_1's l1: 0.186842
[900]	training's l2: 0.00377297	training's l1: 0.0437322	v

[INFO]2019-06-25 16:50:06,872:main:fold 3 valid -2.076177
[INFO]2019-06-25 16:50:06,897:main:CV score: -2.086247


In [0]:
for _, df_score in score_dict.items():
    display(df_score.mean()[0])

### Check training result

In [0]:
# sns.distplot(df_pred['proba'])

In [0]:
def feat_importance(_models, _X, _imp_type='gain'):
    df_imp = pd.DataFrame(index=_X.columns)
    for i, model in enumerate(_models):
        df_imp[i] = model.booster_.feature_importance(importance_type=_imp_type)

    df_imp = df_imp.apply(lambda x: x/sum(x))
    df_imp['imp_mean'] = df_imp[list(range(len(models)))].mean(axis=1)
    df_imp['imp_std'] = df_imp[list(range(len(models)))].std(axis=1)
    sorted_imp = df_imp.sort_values(by='imp_mean', ascending=False)
    return sorted_imp

In [0]:
# imp = feat_importance(model_dict['1JHC'], X, _imp_type='gain')
# imp.head(100)

## Predict

In [0]:
df_test = pd.read_csv(TEST_PATH)
df_strct = pd.read_csv(INPUT + 'structures.csv')

In [0]:
def predict_single(df, strct):
    models = joblib.load(MODEL_PATH)

    df_submit = df[['id']].copy()
    df = preprocess(df, strct, mode='predict')
    X = drop_col(df)
    display(X.head())
    
    X.to_csv('test_prepro.csv', index=False)
    
    y_pred = oof_predict(models, X)
    df_submit['scalar_coupling_constant'] = y_pred
    
    return df_submit

In [0]:
def predict_each_type(df, strct):
    # df = df.head(10000)
    model_dict = joblib.load(MODEL_PATH)
    
    s_type = df['type'].copy()
    df_submit = df[['id']].copy()
    
    df = preprocess(df, strct, mode='predict')
    df = drop_col(df)    
    
    coupling_types = s_type.unique()
    print(coupling_types)
    for coup_type in coupling_types:
        
        models = model_dict[coup_type]
        
        get_logger().info('Starting predict target(%s)' % coup_type)
        is_the_type = (s_type == coup_type)
        df_type = df[is_the_type]
                      
        X = df_type
        X = drop_uneffect_feature(X)        
        
        display(X.head())  
        y_pred = oof_predict(models, X)        
        
        df_submit.loc[is_the_type, 'scalar_coupling_constant'] = y_pred
    
    display(df_submit.head())
    print((df_submit[TARGET].isnull()).sum())
    return df_submit

In [0]:
df_submit = predict_each_type(df_test, df_strct)

[INFO]2019-06-25 16:50:48,461:main:Start preprocess()
[INFO]2019-06-25 16:50:48,462:main:load df_1j
[INFO]2019-06-25 16:50:53,185:main:load df_2jsim
[INFO]2019-06-25 16:51:08,124:main:load df_3jsim


Starting Feature Engineering...


,id,molecule_name,atom_index_0,atom_index_1,type,1j_nbonds,2j_atom_center,2j_area_021,2j_norm_vec_02,2j_norm_vec_12,2j_cos,2j_atom_center_weight,2j_sum_norm_vec,3j_norm_vec_02,3j_norm_vec_13,3j_norm_vec_23,3j_cos_023,3j_cos_231,3j_area_023,3j_area_231,3j_dihedral,3j_atom_center_weight,3j_atom_center,3j_sum_norm_vec,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,dist_x,dist_y,dist_z,type_0,type_1,molecule_couples,molecule_dist_mean,...,molecule_atom_index_0_z_1_std,molecule_atom_index_0_dist_mean,molecule_atom_index_0_dist_mean_diff,molecule_atom_index_0_dist_mean_div,molecule_atom_index_0_dist_max,molecule_atom_index_0_dist_max_diff,molecule_atom_index_0_dist_max_div,molecule_atom_index_0_dist_min,molecule_atom_index_0_dist_min_diff,molecule_atom_index_0_dist_min_div,molecule_atom_index_0_dist_std,molecule_atom_index_0_dist_std_diff,molecule_atom_index_0_dist_std_div,molecule_atom_index_1_dist_mean,molecule_atom_index_1_dist_mean_diff,molecule_atom_index_1_dist_mean_div,molecule_atom_index_1_dist_max,molecule_atom_index_1_dist_max_diff,molecule_atom_index_1_dist_max_div,molecule_atom_index_1_dist_min,molecule_atom_index_1_dist_min_diff,molecule_atom_index_1_dist_min_div,molecule_atom_index_1_dist_std,molecule_atom_index_1_dist_std_diff,molecule_atom_index_1_dist_std_div,molecule_atom_1_dist_mean,molecule_atom_1_dist_min,molecule_atom_1_dist_min_diff,molecule_atom_1_dist_min_div,molecule_atom_1_dist_std,molecule_atom_1_dist_std_diff,molecule_type_0_dist_std,molecule_type_0_dist_std_diff,molecule_type_dist_mean,molecule_type_dist_mean_diff,molecule_type_dist_mean_div,molecule_type_dist_max,molecule_type_dist_min,molecule_type_dist_std,molecule_type_dist_std_diff
0,4658147,dsgdb9nsd_000004,2,0,2JHC,NaN,C,0.000000,1.062099,1.199079,-1.000000,12.01,2.261178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,-1.661639,0.000000,1.000000,C,0.599539,0.000000,1.000000,2.261178,5.112926,0.000000,0.000000,2,JHC,5,1.993966,...,0.000000,2.215518,-0.045660,0.979807,3.323277,1.062099,1.469711,1.062099,-1.199079,0.469711,1.131280,-1.129898,0.500306,1.661639,-0.599539,0.734855,2.261178,0.000000,1.000000,1.062099,-1.199079,0.469711,0.847877,-1.413301,0.374971,1.661639,1.062099,-1.199079e+00,0.469711,0.692289,-1.568890,0.000000,-2.261178,2.261178,0.000000,1.000000,2.261178,2.261178,0.000000,-2.261178
1,4658148,dsgdb9nsd_000004,2,1,1JHC,2.0,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,-1.661639,0.000000,1.000000,C,-0.599539,0.000000,1.000000,1.062099,1.128054,0.000000,0.000000,1,JHC,5,1.993966,...,0.000000,2.215518,1.153419,2.085981,3.323277,2.261178,3.128971,1.062099,0.000000,1.000000,1.131280,0.069181,1.065136,1.661639,0.599539,1.564485,2.261178,1.199079,2.128971,1.062099,0.000000,1.000000,0.847877,-0.214222,0.798303,1.661639,1.062099,0.000000e+00,1.000000,0.692289,-0.369811,0.000000,-1.062099,1.062099,0.000000,1.000000,1.062099,1.062099,0.000000,-1.062099
2,4658149,dsgdb9nsd_000004,2,3,3JHH,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1.062099,1.062099,1.199079,-1.000000,-1.000000,0.00000,0.000000,NaN,24.02,CC,3.323277,H,-1.661639,0.000000,1.000000,H,1.661639,0.000000,1.000000,3.323277,11.044171,0.000000,0.000000,3,JHH,5,1.993966,...,0.000000,2.215518,-1.107759,0.666667,3.323277,0.000000,1.000000,1.062099,-2.261178,0.319594,1.131280,-2.191997,0.340411,3.323277,0.000000,1.000000,3.323277,0.000000,1.000000,3.323277,0.000000,1.000000,NaN,NaN,NaN,3.323277,3.323277,0.000000e+00,1.000000,NaN,NaN,NaN,NaN,3.323277,0.000000,1.000000,3.323277,3.323277,NaN,NaN
3,4658150,dsgdb9nsd_000004,3,0,1JHC,2.0,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,1.661639,0.000000,1.000000,C,0.599539,0.000000,1.000000,1.062099,1.128054,0.000000,0.000000,1,JHC,5,1.993966,...,0.000000,1.661639,0.599539,1.564485,2.261178,1.199079,2.128971,1.062099,0.000000,1.000000,0.847877,-0.214222,0.798303,1.661639,0.599539,1.564485,2.261178,1.199079,2.128971,1.062099,0.000000,1.000000,0.847877,-0.214222,0.798303,1.661639,1.062099,0.000000e+00,1.000000,0.6

,id,molecule_name,atom_index_0,atom_index_1,type,1j_nbonds,2j_atom_center,2j_area_021,2j_norm_vec_02,2j_norm_vec_12,2j_cos,2j_atom_center_weight,2j_sum_norm_vec,3j_norm_vec_02,3j_norm_vec_13,3j_norm_vec_23,3j_cos_023,3j_cos_231,3j_area_023,3j_area_231,3j_dihedral,3j_atom_center_weight,3j_atom_center,3j_sum_norm_vec,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,dist_x,dist_y,dist_z,type_0,type_1,molecule_couples,molecule_dist_mean,...,molecule_atom_index_0_z_1_std,molecule_atom_index_0_dist_mean,molecule_atom_index_0_dist_mean_diff,molecule_atom_index_0_dist_mean_div,molecule_atom_index_0_dist_max,molecule_atom_index_0_dist_max_diff,molecule_atom_index_0_dist_max_div,molecule_atom_index_0_dist_min,molecule_atom_index_0_dist_min_diff,molecule_atom_index_0_dist_min_div,molecule_atom_index_0_dist_std,molecule_atom_index_0_dist_std_diff,molecule_atom_index_0_dist_std_div,molecule_atom_index_1_dist_mean,molecule_atom_index_1_dist_mean_diff,molecule_atom_index_1_dist_mean_div,molecule_atom_index_1_dist_max,molecule_atom_index_1_dist_max_diff,molecule_atom_index_1_dist_max_div,molecule_atom_index_1_dist_min,molecule_atom_index_1_dist_min_diff,molecule_atom_index_1_dist_min_div,molecule_atom_index_1_dist_std,molecule_atom_index_1_dist_std_diff,molecule_atom_index_1_dist_std_div,molecule_atom_1_dist_mean,molecule_atom_1_dist_min,molecule_atom_1_dist_min_diff,molecule_atom_1_dist_min_div,molecule_atom_1_dist_std,molecule_atom_1_dist_std_diff,molecule_type_0_dist_std,molecule_type_0_dist_std_diff,molecule_type_dist_mean,molecule_type_dist_mean_diff,molecule_type_dist_mean_div,molecule_type_dist_max,molecule_type_dist_min,molecule_type_dist_std,molecule_type_dist_std_diff
2505532,7163679,dsgdb9nsd_133885,14,7,1JHC,4.0,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,0.013699,1.199431,-1.680192,C,-0.095929,0.380424,-0.972098,1.088204,0.012018,0.670772,0.501398,1,JHC,52,2.499017,...,0.882245,2.537945,1.449741,2.332233,3.367225,2.279021,3.094296,1.088204,0.000000,1.000000,0.688326,-0.399878,0.632534,2.687623,1.599419,2.469779,3.378975,2.290771,3.105093,1.088204,0.000000,1.000000,0.814510,-0.273695,0.748490,2.465317,1.080482,-7.721802e-03,0.992904,0.771098,-0.317106,0.005294,-1.082910,1.085940,-0.002265,0.997919,1.091826,1.080482,0.005294,-1.082910
2505533,7163680,dsgdb9nsd_133885,14,8,2JHC,NaN,C,1.402630,1.088204,1.503272,-0.514613,12.01,2.591476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,H,0.013699,1.199431,-1.680192,C,0.816694,-0.813067,-1.022360,2.264441,0.644802,4.050146,0.432744,2,JHC,52,2.499017,...,0.882245,2.537945,0.273504,1.120782,3.367225,1.102785,1.487001,1.088204,-1.176237,0.480562,0.688326,-1.576114,0.303972,2.268936,0.004495,1.001985,3.367229,1.102789,1.487003,1.080482,-1.183958,0.477152,0.809831,-1.454609,0.357630,2.465317,1.080482,-1.183958e+00,0.477152,0.771098,-1.493343,0.113677,-2.150764,2.299965,0.035524,1.015688,2.324550,2.264441,0.021884,-2.242557
2505534,7163681,dsgdb9nsd_133885,14,15,3JHH,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1.088204,1.080482,1.503272,-0.514613,-0.595536,1.402630,1.304814,0.839732,24.02,CC,3.671959,H,0.013699,1.199431,-1.680192,H,1.260745,-1.246754,-1.906767,2.755046,1.555125,5.983817,0.051336,3,JHH,52,2.499017,...,0.882245,2.537945,-0.217101,0.921199,3.367225,0.612179,1.222203,1.088204,-1.666842,0.394986,0.688326,-2.066720,0.249842,2.744210,-0.010836,0.996067,2.755046,0.000000,1.000000,2.737267,-0.017779,0.993547,0.009507,-2.745539,0.003451,2.589861,1.811174,-9.438719e-01,0.657403,0.381557,-2.373489,0.329958,-2.425088,2.745599,-0.009447,0.996571,2.755049,2.737267,0.008715,-2.746331
2505535,7163682,dsgdb9nsd_133885,15,1,3JHN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1.080482,1.563097,1.503272,-0.595536,-0.437273,1.304814,2.113208,-0.403965,24.02,CC,4.146852,H,1.260745,-1.246754,-1.906767,N,-1.564594,0.099125,-0.516941,3.424258,7.982540,1.811389,1.931616,3,JHN,52,2.499017,...,0.700067,2.585814,-0.838445,0.755146,3.424258,0.000000,1.000000,1.080482,-2.343776,0.315538,0.846251,-2.578007,0.247134,

[INFO]2019-06-25 16:51:57,583:main:loading encoder from ./analysis/mole/data/preprocess/le.pkl
[INFO]2019-06-25 16:52:00,682:main:['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type', '1j_nbonds', '2j_atom_center', '2j_area_021', '2j_norm_vec_02', '2j_norm_vec_12', '2j_cos', '2j_atom_center_weight', '2j_sum_norm_vec', '3j_norm_vec_02', '3j_norm_vec_13', '3j_norm_vec_23', '3j_cos_023', '3j_cos_231', '3j_area_023', '3j_area_231', '3j_dihedral', '3j_atom_center_weight', '3j_atom_center', '3j_sum_norm_vec', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'dist', 'dist_x', 'dist_y', 'dist_z', 'type_0', 'type_1', 'molecule_couples', 'molecule_dist_mean', 'molecule_dist_min', 'molecule_dist_max', 'atom_0_couples_count', 'atom_1_couples_count', 'molecule_atom_index_0_x_1_std', 'molecule_atom_index_0_y_1_mean', 'molecule_atom_index_0_y_1_mean_diff', 'molecule_atom_index_0_y_1_mean_div', 'molecule_atom_index_0_y_1_max', 'molecule_atom_index_0_y_1_max_diff', 'molecule_atom

...

[INFO]2019-06-25 17:04:20,930:main:Start oof_predict
[INFO]2019-06-25 17:04:20,932:main:prediction: 0
[INFO]2019-06-25 17:04:32,782:main:prediction: 1
[INFO]2019-06-25 17:04:44,434:main:prediction: 2
[INFO]2019-06-25 17:04:56,147:main:Finish oof_predict
[INFO]2019-06-25 17:04:56,189:main:Starting predict target(2JHN)


,2j_atom_center,2j_area_021,2j_norm_vec_02,2j_norm_vec_12,2j_cos,2j_atom_center_weight,2j_sum_norm_vec,dist,dist_x,dist_y,dist_z,molecule_couples,molecule_dist_mean,molecule_dist_min,molecule_dist_max,atom_0_couples_count,atom_1_couples_count,molecule_atom_index_0_x_1_std,molecule_atom_index_0_y_1_mean,molecule_atom_index_0_y_1_mean_diff,molecule_atom_index_0_y_1_mean_div,molecule_atom_index_0_y_1_max,molecule_atom_index_0_y_1_max_diff,molecule_atom_index_0_y_1_std,molecule_atom_index_0_z_1_std,molecule_atom_index_0_dist_mean,molecule_atom_index_0_dist_mean_diff,molecule_atom_index_0_dist_mean_div,molecule_atom_index_0_dist_max,molecule_atom_index_0_dist_max_diff,molecule_atom_index_0_dist_max_div,molecule_atom_index_0_dist_min,molecule_atom_index_0_dist_min_diff,molecule_atom_index_0_dist_min_div,molecule_atom_index_0_dist_std,molecule_atom_index_0_dist_std_diff,molecule_atom_index_0_dist_std_div,molecule_atom_index_1_dist_mean,molecule_atom_index_1_dist_mean_diff,molecule_atom_index_1_dist_mean_div,molecule_atom_index_1_dist_max,molecule_atom_index_1_dist_max_diff,molecule_atom_index_1_dist_max_div,molecule_atom_index_1_dist_min,molecule_atom_index_1_dist_min_diff,molecule_atom_index_1_dist_min_div,molecule_atom_index_1_dist_std,molecule_atom_index_1_dist_std_diff,molecule_atom_index_1_dist_std_div,molecule_atom_1_dist_mean,molecule_atom_1_dist_min,molecule_atom_1_dist_min_diff,molecule_atom_1_dist_min_div,molecule_atom_1_dist_std,molecule_atom_1_dist_std_diff,molecule_type_0_dist_std,molecule_type_0_dist_std_diff,molecule_type_dist_mean,molecule_type_dist_mean_diff,molecule_type_dist_mean_div,molecule_type_dist_max,molecule_type_dist_min,molecule_type_dist_std,molecule_type_dist_std_diff
141,0,1.480469,1.095703,1.451172,-0.364258,12.007812,2.546875,2.113281,0.328613,3.720703,0.413330,22,2.082031,1.006836,3.402344,6,5,0.498779,0.660645,0.622070,17.239294,1.824219,1.785156,1.154297,0.781250,2.082031,-0.030411,0.985840,3.191406,1.078125,1.510742,1.095703,-1.017578,0.518555,0.719727,-1.392578,0.340576,1.867188,-0.245728,0.883789,2.113281,0.000013,1.000000,1.006836,-1.105469,0.476562,0.481689,-1.630859,0.228027,1.867188,1.006836,-1.105469,0.476562,0.481689,-1.630859,0.154541,-1.958008,2.082031,-0.030807,0.985352,2.113281,2.044922,0.035919,-2.076172
147,0,1.480469,1.095703,1.451172,-0.364258,12.007812,2.546875,2.113281,1.373047,2.962891,0.126587,22,2.082031,1.006836,3.402344,5,5,0.166748,0.440674,0.402344,11.500521,1.824219,1.785156,1.141602,0.827637,2.142578,0.029770,1.013672,3.191406,1.078125,1.510742,1.095703,-1.017578,0.518555,0.787598,-1.325195,0.372803,1.867188,-0.245850,0.883789,2.113281,0.000000,1.000000,1.006836,-1.105469,0.476562,0.481689,-1.630859,0.228027,1.867188,1.006836,-1.105469,0.476562,0.481689,-1.630859,0.154541,-1.958008,2.082031,-0.030807,0.985352,2.113281,2.044922,0.035919,-2.076172
152,0,1.510742,1.090820,1.451172,-0.296387,12.007812,2.541016,2.056641,0.001970,3.187500,1.041992,22,2.082031,1.006836,3.402344,4,5,0.191040,0.094788,0.056488,2.474538,1.478516,1.440430,0.969727,0.818359,2.158203,0.100830,1.048828,2.968750,0.912598,1.443359,1.090820,-0.966797,0.530273,0.803223,-1.253906,0.390381,1.867188,-0.190308,0.907715,2.113281,0.055481,1.027344,1.006836,-1.049805,0.489502,0.481689,-1.575195,0.234131,1.867188,1.006836,-1.049805,0.489502,0.481689,-1.575195,0.154541,-1.902344,2.082031,0.024673,1.011719,2.113281,2.044922,0.035919,-2.021484
160,0,1.402344,1.108398,1.359375,-0.366211,12.007812,2.468750,2.044922,0.132080,3.207031,0.842285,22,2.082031,1.006836,3.402344,3,5,0.222900,0.282959,0.244507,7.383074,1.478516,1.440430,1.094727,0.653809,2.185547,0.140625,1.068359,3.402344,1.358398,1.664062,1.108398,-0.936523,0.541992,1.154297,-0.891113,0.564453,1.867188,-0.178101,0.913086,2.113281,0.067749,1.033203,1.006836,-1.038086,0.492432,0.481689,-1.563477,0.235596,1.867188,1.006836,-1.038086,0.492432,0.481689,-1.563477,0.154541,-1.890625,2.082031,0.036926,1.017578,2.113281,2.044922,0.035919,-2.009766
264,0,1.475586,1.10

[INFO]2019-06-25 17:04:57,941:main:Start oof_predict
[INFO]2019-06-25 17:04:57,943:main:prediction: 0
[INFO]2019-06-25 17:05:07,599:main:prediction: 1
[INFO]2019-06-25 17:05:17,001:main:prediction: 2
[INFO]2019-06-25 17:05:26,299:main:Finish oof_predict


,id,scalar_coupling_constant
0,4658147,19.269504
1,4658148,196.538797
2,4658149,2.000638
3,4658150,190.612833
4,4658151,17.258757


0


In [0]:
display(df_submit.head())
df_submit.to_csv('submission.csv', index=False)

,id,scalar_coupling_constant
0,4658147,19.269504
1,4658148,196.538797
2,4658149,2.000638
3,4658150,190.612833
4,4658151,17.258757


In [0]:
df_submit.shape

(2505542, 2)